In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import math

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

In [ ]:
def diff_norm_dataset(sensor_data0):
    a = sensor_data0
    b = a[:-1]
    a = a[1:]-b
    c = np.array([0]+a.tolist())
    mean = c.mean()
    print("mean is: ",mean)
    std = c.std()
    print("std is ",std)
    c = (c-mean)/std

    extream_num = 0
    print("extream boundary is: ", std*alpha_extreams)
    total_num = len(c)
    for i in range(total_num):
        if(abs(c[i])>alpha_extreams):
            extream_num += 1
    print(" the total number is: ", total_num)
    print(" the number of extreams is: ", extream_num, " the ratio is: ", extream_num/total_num)

    return c, mean, std

In [ ]:
import random
import pandas as pd
import numpy as np

def read_dataset(sensor_id, start_point, train_point):
    # read sensor data to vector
    if(str(sensor_id)[0]=='4'):
        trainX = pd.read_csv('./data'+'/reservoir_stor_'+str(sensor_id)+'_sof24.tsv', sep='\t')
    elif(str(sensor_id)[0]=='6'):
        trainX = pd.read_csv('./data'+'/raingauge_byhour_'+str(sensor_id)+'_sof.tsv', sep='\t')
    else:
        print("Error: the support sensor type is 4***or 6****.")
        sys.exit()
    start_num = trainX[trainX["TSC_TSTAMP_UTC"]==start_point].index.values[0]
    print("for sensor ", sensor_id,"start_num is: ", start_num)
    idx_num = 0
    print(len(trainX))
    print(trainX[:3])

    #foot label of train_end
    train_end = trainX[trainX["TSC_TSTAMP_UTC"]==train_point].index.values[0] - start_num 
    print("train_end is : ", train_end)
    
    #the whole dataset to be preprocessed
    train_x = trainX[start_num:] 
    print(len(train_x))
    sensor_data = np.array(train_x["TSC_VALUE_F"])
    sensor_time = np.array(train_x["TSC_TSTAMP_UTC"])
    
    
    return trainX, sensor_data, sensor_time, start_num, train_end
 

In [ ]:
def gen_samples(start): # generate 24 points in the year start from  start_time_id
    
    extreme_num = 10 # at least extreme_num points have extreme values in the forecasting zone
    extreme_total = 0
    random_total = 0
        
    gen_set = []
    r_set = []
   
    while((extreme_total<extreme_num) or (random_total<24-extreme_num) ):
        
        i = random.randint(0, ll-1)
            
        if i not in gen_set:
            label = np.array(class_label[start+i:(start+i+72)])

            if(label.sum()>=1 and extreme_total<extreme_num):
                extreme_total += 1;
                gen_set.append(i)
                r_set.append(sensor_time[start+i])
                print("Having extreme point in the seq start with: ",sensor_time[start+i])
            elif random_total<(24-extreme_num):
                random_total += 1;
                gen_set.append(i)
                r_set.append(sensor_time[start+i])
                if label.sum()>=1 :
                    print("Having extreme point in the seq start with: ",sensor_time[start+i])
                
    return r_set

In [ ]:
sensor_ids = [4001]

for sensor_id in sensor_ids:
    
    start_point = '1980-12-31 14:30:00'
    train_end = 0 
    train_point = '2019-12-31 14:30:00'
    print("Begin!! Sensor id is :---------------------", sensor_id)
    alpha_extreams = 1.5 # normal/extreme boundary, \epsilon=1.5
    
    trainX, sensor_data, sensor_time, start_num, train_end = read_dataset(sensor_id=sensor_id, start_point=start_point, train_point=train_point)    
    sensor_data_norm, mean, std = diff_norm_dataset(sensor_data)
    # 2 years for val, 2 years for test
    val1_start_id = trainX[trainX["TSC_TSTAMP_UTC"]=='2014-01-01 14:30:00'].index.values[0] - start_num      
    val1_end_id = trainX[trainX["TSC_TSTAMP_UTC"]=='2014-12-28 14:30:00'].index.values[0] - start_num  
    val2_start_id = trainX[trainX["TSC_TSTAMP_UTC"]=='2016-01-01 14:30:00'].index.values[0] - start_num      
    val2_end_id = trainX[trainX["TSC_TSTAMP_UTC"]=='2016-12-28 14:30:00'].index.values[0] - start_num 
    test1_start_id = trainX[trainX["TSC_TSTAMP_UTC"]=='2017-01-01 14:30:00'].index.values[0] - start_num      
    test1_end_id = trainX[trainX["TSC_TSTAMP_UTC"]=='2017-12-28 14:30:00'].index.values[0] - start_num 
    test2_start_id = trainX[trainX["TSC_TSTAMP_UTC"]=='2018-01-01 14:30:00'].index.values[0] - start_num      
    test2_end_id = trainX[trainX["TSC_TSTAMP_UTC"]=='2018-12-28 14:30:00'].index.values[0] - start_num 
    
    ll = min(val1_end_id-val1_start_id, val2_end_id-val2_start_id, test1_end_id-test1_start_id, test2_end_id-test2_start_id)    

    class_label = []
    for i in range(len(sensor_data_norm)):
        if abs(sensor_data_norm[i])>alpha_extreams:
            class_label.append(1)
        else:
            class_label.append(0)
            
    val1 = gen_samples(val1_start_id)
    val2 = gen_samples(val2_start_id)
    test1 = gen_samples(test1_start_id)
    test2 = gen_samples(test2_start_id)
    
    val = val1 + val2
    df = pd.DataFrame(val, columns=['Hold Out Start'])  
    df.to_csv('./data/'+str(sensor_id)+'_validation_timestamps_24avg.tsv', sep='\t')
    
    test = test1 + test2
    df = pd.DataFrame(test, columns=['Start'])  
    df.to_csv('./data/'+str(sensor_id)+'_test_timestamps_24avg.tsv', sep='\t')
    